In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [39]:
def dec2bin(x, n=7):
    """
    creating a binary list of integer non-negative x
    """
    u = 2 ** n - 1
    x = int(x)
    assert x >= 0, 'Input value x must be non-negative'
    assert x <= u, f'Input value x with n = {n} must be less than {u}'
    
    y = []
    if x == 0:
        for i in range(n):
            y.append(0)
    else:
        while x != 1:
            y.append(x % 2)
            x = x // 2
        y.append(x)
        delta = n - len(y)
        for i in range(delta):
            y.append(0)
        y.reverse()
    return y

In [40]:
def get_all_X(n_inputs):
    """
    for given number of variables returns
    all binary combinations of these variables
    """
    X = []
    for i in range(2**n_inputs):
        X += [dec2bin(i, n=n_inputs)]
    X = np.array(X)
    return X

In [41]:
def random_boolfunc(n_inputs):
    """
    for given number of variables returns
    the truth table, where output is random binary vector
    """
    X = get_all_X(n_inputs)
    y = np.random.randint(0, 2, size=(2**n_inputs, 1))
    return X, y
   
def _and(n_inputs=2):
    """
    for given number of variables returns
    the truth table of AND logical gate
    """
    X = get_all_X(n_inputs)
    y = np.append(
        np.zeros(shape=(2 ** n_inputs - 1, 1), dtype=int),
        np.array([[1]]),
        axis=0
    )
    return X, y
    
def _or(n_inputs=2):
    """
    for given number of variables returns
    the truth table of OR logical gate
    """
    X = get_all_X(n_inputs)
    y = np.append(
        np.array([[0]]),
        np.ones(shape=(2 ** n_inputs - 1, 1), dtype=int),
        axis=0
    )
    return X, y

def _xor(n_inputs=2):
    """
    for given number of variables returns
    the truth table of XOR logical gate
    """
    X = get_all_X(n_inputs)
    y = (np.sum(X, axis=1) % 2).reshape(-1, 1)
    return X, y

def to_dataframe(X, y):
    """
    for the truth table in form of two arrays 
    X [2 ** n_inputs, n_inputs] and y [2 ** n_inputs, 1]
    combine it to the form of Pandas DataFrame
    """
    data=np.concatenate((X, y), axis=1)
    n_inputs = X.shape[1]
    return pd.DataFrame(data=data, columns=[f'x{i}' for i in range(n_inputs, 0, -1)] + ['y'])

In [42]:
n = 2

In [43]:
%%time
X, y = random_boolfunc(n_inputs=n)
X, y = torch.Tensor(X), torch.Tensor(y)
X, y

Wall time: 26.9 ms


(tensor([[0., 0.],
         [0., 1.],
         [1., 0.],
         [1., 1.]]),
 tensor([[0.],
         [0.],
         [1.],
         [1.]]))

In [164]:
X, y = _xor()
X, y = torch.Tensor(X), torch.Tensor(y)
X, y

(tensor([[0., 0.],
         [0., 1.],
         [1., 0.],
         [1., 1.]]),
 tensor([[0.],
         [1.],
         [1.],
         [0.]]))

In [129]:
class Neuron(nn.Module):
    def __init__(self, n_inputs, activation=nn.Sigmoid()):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, 1)
        #self.fc2 = nn.Linear(2,1)
        self.activation = activation
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        #x = self.fc2(x)
        #x = self.activation(x)
        return x

In [137]:
model = Neuron(n_inputs=n)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.MSELoss()
n_epochs = 1000
for i in range(1, n_epochs+1):
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'epoch {i}: loss {loss:.2f}')

epoch 10: loss 0.69
epoch 20: loss 0.69
epoch 30: loss 0.69
epoch 40: loss 0.69
epoch 50: loss 0.69
epoch 60: loss 0.69
epoch 70: loss 0.69
epoch 80: loss 0.69
epoch 90: loss 0.69
epoch 100: loss 0.69
epoch 110: loss 0.69
epoch 120: loss 0.69
epoch 130: loss 0.69
epoch 140: loss 0.69
epoch 150: loss 0.69
epoch 160: loss 0.69
epoch 170: loss 0.69
epoch 180: loss 0.69
epoch 190: loss 0.69
epoch 200: loss 0.69
epoch 210: loss 0.69
epoch 220: loss 0.69
epoch 230: loss 0.69
epoch 240: loss 0.69
epoch 250: loss 0.69
epoch 260: loss 0.69
epoch 270: loss 0.69
epoch 280: loss 0.69
epoch 290: loss 0.69
epoch 300: loss 0.69
epoch 310: loss 0.69
epoch 320: loss 0.69
epoch 330: loss 0.69
epoch 340: loss 0.69
epoch 350: loss 0.69
epoch 360: loss 0.69
epoch 370: loss 0.69
epoch 380: loss 0.69
epoch 390: loss 0.69
epoch 400: loss 0.69
epoch 410: loss 0.69
epoch 420: loss 0.69
epoch 430: loss 0.69
epoch 440: loss 0.69
epoch 450: loss 0.69
epoch 460: loss 0.69
epoch 470: loss 0.69
epoch 480: loss 0.69
e

In [142]:
pred, y

(tensor([[0.5000],
         [0.5000],
         [0.5000],
         [0.5000]], grad_fn=<SigmoidBackward0>),
 tensor([[0.],
         [1.],
         [1.],
         [0.]]))

In [150]:
X_new = X[[0,3]]
y_new = y[[0,3]]

In [146]:
X_new

tensor([[0., 0.],
        [1., 1.]])

In [151]:
model2 = Neuron(n_inputs=n)
optimizer = torch.optim.Adam(model2.parameters(), lr = 0.1)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.MSELoss()
n_epochs = 1000
for i in range(1, n_epochs+1):
        optimizer.zero_grad()
        pred2 = model2(X_new)
        loss = criterion(pred2, y_new)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'epoch {i}: loss {loss:.2f}')

epoch 10: loss 0.25
epoch 20: loss 0.11
epoch 30: loss 0.07
epoch 40: loss 0.04
epoch 50: loss 0.03
epoch 60: loss 0.03
epoch 70: loss 0.02
epoch 80: loss 0.02
epoch 90: loss 0.02
epoch 100: loss 0.01
epoch 110: loss 0.01
epoch 120: loss 0.01
epoch 130: loss 0.01
epoch 140: loss 0.01
epoch 150: loss 0.01
epoch 160: loss 0.01
epoch 170: loss 0.01
epoch 180: loss 0.01
epoch 190: loss 0.01
epoch 200: loss 0.01
epoch 210: loss 0.01
epoch 220: loss 0.00
epoch 230: loss 0.00
epoch 240: loss 0.00
epoch 250: loss 0.00
epoch 260: loss 0.00
epoch 270: loss 0.00
epoch 280: loss 0.00
epoch 290: loss 0.00
epoch 300: loss 0.00
epoch 310: loss 0.00
epoch 320: loss 0.00
epoch 330: loss 0.00
epoch 340: loss 0.00
epoch 350: loss 0.00
epoch 360: loss 0.00
epoch 370: loss 0.00
epoch 380: loss 0.00
epoch 390: loss 0.00
epoch 400: loss 0.00
epoch 410: loss 0.00
epoch 420: loss 0.00
epoch 430: loss 0.00
epoch 440: loss 0.00
epoch 450: loss 0.00
epoch 460: loss 0.00
epoch 470: loss 0.00
epoch 480: loss 0.00
e

In [152]:
pred2, y_new

(tensor([[9.6581e-04],
         [1.2281e-05]], grad_fn=<SigmoidBackward0>),
 tensor([[0.],
         [0.]]))

In [139]:
list(model.parameters())

[Parameter containing:
 tensor([[ 6.5637e-08, -3.5102e-08]], requires_grad=True),
 Parameter containing:
 tensor([-1.0935e-07], requires_grad=True)]

In [169]:
print(*[1, 2, 3])

1 2 3


In [171]:
from sympy.logic.boolalg import ANFform

In [174]:
from sympy.logic.boolalg import ANFform
from sympy.abc import x, y, z
ANFform([x], [1, 0])

x ^ True

In [173]:
ANFform([x, y], [0, 1, 1, 1])

x ^ y ^ (x & y)

In [176]:
ANFform([x, y, z], [1, 0, 0, 0, 1, 0, 1, 1])

y ^ z ^ True ^ (x & y) ^ (y & z)

In [189]:
from sympy.logic.boolalg import anf_coeffs, bool_monomial, Xor
from sympy.abc import a, b, c
truthvalues = [int(i) for i in list('10001011')]
coeffs = anf_coeffs(truthvalues)
coeffs


[1, 1, 1, 1, 0, 0, 1, 0]

In [190]:
polynomial = Xor(*[
    bool_monomial(k, [a, b, c])
    for k, coeff in enumerate(coeffs) if coeff == 1
])
polynomial

~(b ^ c ^ (a & b) ^ (b & c))